# Practice notebook for hypothesis tests using NHANES data

This notebook will give you the opportunity to perform some hypothesis tests with the NHANES data that are similar to
what was done in the week 3 case study notebook.

You can enter your code into the cells that say "enter your code here", and you can type responses to the questions into the cells that say "Type Markdown and Latex".

Note that most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [45]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np
import scipy.stats.distributions as dist

da = pd.read_csv("nhanes_2015_2016.csv")
print(da.columns)
da.head()

Index(['SEQN', 'ALQ101', 'ALQ110', 'ALQ130', 'SMQ020', 'RIAGENDR', 'RIDAGEYR',
       'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'WTINT2YR',
       'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BPXSY2',
       'BPXDI2', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 'BMXARMC',
       'BMXWAIST', 'HIQ210'],
      dtype='object')


,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


## Question 1

Conduct a hypothesis test (at the 0.05 level) for the null hypothesis that the proportion of women who smoke is equal to the proportion of men who smoke.

In [74]:
da["SMQ020x"] = da.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})  
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})
dx = da[["SMQ020x", "RIAGENDRx"]].dropna()
dx_table = da.groupby("RIAGENDRx")["SMQ020x"].agg([lambda x: np.mean(x=="Yes"), np.size])
dx_table.columns = ["p", "n"]
print(dx_table)
p_comb = np.mean(da.SMQ020x == "Yes")
se = np.sqrt((p_comb * (1 - p_comb)) * (1/dx_table.n.Female + 1/dx_table.n.Male))
test_stat = (dx_table.p.Female - dx_table.p.Male)/se
pvalue = 2*dist.norm.cdf(-np.abs(test_stat))
print(pvalue, test_stat, p_comb, se)
dx_female = dx.loc[dx.RIAGENDRx == "Female", "SMQ020x"].replace({"Yes": 1, "No": 2})
dx_male = dx.loc[dx.RIAGENDRx == "Male", "SMQ020x"].replace({"Yes": 1, "No": 2})
print(sm.stats.ttest_ind(dx_female, dx_male))

                  p     n
RIAGENDRx                
Female     0.304435  2976
Male       0.512142  2759
1.0202277513491146e-57 -16.01400681759787 0.404359197907585 0.012970307741140625
(16.420585558984424, 3.032088786691548e-59, 5723.0)


__Q1a.__ Write 1-2 sentences explaining the substance of your findings to someone who does not know anything about statistical hypothesis tests.

P > 0.05, NOT reject null.  

__Q1b.__ Construct three 95% confidence intervals: one for the proportion of women who smoke, one for the proportion of men who smoke, and one for the difference in the rates of smoking between women and men.

In [43]:
dx_table2 = pd.crosstab(dx.RIAGENDRx, dx.SMQ020x)
dx_table2 ["total"] = dx_table2.No + dx_table2.Yes
print(dx_table2)
print(sm.stats.proportion_confint(906,2972))
print(sm.stats.proportion_confint(1413,2753))
print(sm.stats.proportion_confint(906+1413,2972+2753))

SMQ020x      No   Yes  total
RIAGENDRx                   
Female     2066   906   2972
Male       1340  1413   2753
(0.2882949879861214, 0.32139545615923526)
(0.49458749263718593, 0.5319290347874418)
(0.3923492963048936, 0.41778170806191856)


__Q1c.__ Comment on any ways in which the confidence intervals that you found in part b reinforce, contradict, or add support to the hypothesis test conducted in part a.

## Question 2

Partition the population into two groups based on whether a person has graduated college or not, using the educational attainment variable [DMDEDUC2](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDEDUC2).  Then conduct a test of the null hypothesis that the average heights (in centimeters) of the two groups are equal.  Next, convert the heights from centimeters to inches, and conduct a test of the null hypothesis that the average heights (in inches) of the two groups are equal.

In [75]:
da["DMDEDUC2x"] = da.DMDEDUC2.replace({1:"No", 2:"No", 3:"No", 4:"No", 
                                       5:"Yes", 7: np.nan, 9: np.nan})
de = da[["BMXHT", "DMDEDUC2x"]].dropna()
de_yes = de.loc[de.DMDEDUC2x == "Yes", "BMXHT"]
de_no = de.loc[de.DMDEDUC2x == "No", "BMXHT"]
print(sm.stats.ttest_ind(de_yes, de_no))

de_table = de.groupby("DMDEDUC2x")["BMXHT"].agg([np.mean, np.size])
de_table.columns = ["mu","n"]
print(de_table)
test_stats = (de_table.mu.Yes - de_table.mu.No)/(de.BMXHT.std()* np.sqrt(1/de_table.n.Yes + 1/de_table.n.No))
pvalue = 2*dist.norm.cdf(-np.abs(test_stats))
print(test_stats,pvalue)

(7.569245765210298, 4.3868465661619117e-14, 5412.0)
                   mu       n
DMDEDUC2x                    
No         165.466519  4059.0
Yes        167.855351  1355.0
7.530191242144503 5.0666060798098924e-14


__Q2a.__ Based on the analysis performed here, are you confident that people who graduated from college have a different average height compared to people who did not graduate from college?

__Q2b:__ How do the results obtained using the heights expressed in inches compare to the results obtained using the heights expressed in centimeters?

## Question 3

Conduct a hypothesis test of the null hypothesis that the average BMI for men between 30 and 40 is equal to the average BMI for men between 50 and 60.  Then carry out this test again after log transforming the BMI values.

In [91]:
db_male = da.loc[(da.RIDAGEYR <= 40) & (da.RIDAGEYR >=30) & (da.RIAGENDRx == "Male"), "BMXBMI"].dropna()
db_female = da.loc[(da.RIDAGEYR <=60) & (da.RIDAGEYR >=50) & (da.RIAGENDRx == "Female"), "BMXBMI"].dropna()
print(sm.stats.ttest_ind(db_male, db_female))
print(sm.stats.ztest(db_male, db_female))

(-2.6849622759647125, 0.007374552223632632, 994.0)
(-2.6849622759647125, 0.007253799305501502)


__Q3a.__ How would you characterize the evidence that mean BMI differs between these age bands, and how would you characterize the evidence that mean log BMI differs between these age bands?

In [95]:
db_male.mean(), db_female.mean()

(29.61372950819672, 30.84507874015748)

## Question 4

Suppose we wish to compare the mean BMI between college graduates and people who have not graduated from college, focusing on women between the ages of 30 and 40.  First, consider the variance of BMI within each of these subpopulations using graphical techniques, and through the estimated subpopulation variances.  Then, calculate pooled and unpooled estimates of the standard error for the difference between the mean BMI in the two populations being compared.  Finally, test the null hypothesis that the two population means are equal, using each of the two different standard errors.

In [ ]:
# insert your code here

__Q4a.__ Comment on the strength of evidence against the null hypothesis that these two populations have equal mean BMI.

__Q4b.__ Comment on the degree to which the two populations have different variances, and on the extent to which the results using different approaches to estimating the standard error of the mean difference give divergent results.

## Question 5

Conduct a test of the null hypothesis that the first and second diastolic blood pressure measurements within a subject have the same mean values.

In [ ]:
# insert your code here

__Q5a.__ Briefly describe your findings for an audience that is not familiar with statistical hypothesis testing.

__Q5b.__ Pretend that the first and second diastolic blood pressure measurements were taken on different people.  Modfify the analysis above as appropriate for this setting.

In [ ]:
# insert your code here

__Q5c.__ Briefly describe how the approaches used and the results obtained in the preceeding two parts of the question differ.